In [ ]:
import os
import nni
import inspect
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import datetime as dt
import lightgbm as lgb
from tqdm import tqdm
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split,cross_val_score,cross_validate,KFold,GridSearchCV
from sklearn.preprocessing import MinMaxScaler,StandardScaler,OneHotEncoder,PolynomialFeatures
from sklearn.decomposition import PCA,TruncatedSVD,NMF
from sklearn.manifold import TSNE
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,mean_squared_error
from sklearn import svm,linear_model,tree
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.svm import SVC
from sklearn.neural_network import MLPRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression,SGDClassifier,LinearRegression,Lasso,BayesianRidge
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,ExtraTreesClassifier
from sklearn.gaussian_process import GaussianProcessClassifier,GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.pipeline import Pipeline,make_pipeline
import joblib

# 数据预处理

In [ ]:
#准备文件夹保存数据
local_path=os.getcwd()
if not os.path.exists(local_path+'\\blokagedata'):
    os.mkdir(local_path+'\\blokagedata')
print(local_path+'\\blokagedata')
#读取模型数据
traindata=pd.read_csv(local_path+'\\blokagedata\\sample_0123456789.csv',parse_dates=[0])
trainlabel=pd.read_csv(local_path+'\\blokagedata\\y_0123456789.csv',parse_dates=[0])
#改个名字
x=traindata.drop(['32','205','375','408','414'],axis=1)
y=np.array(trainlabel).astype(int)
#划分训练集和测试集
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1,stratify=y)
#归一化
scaler1=MinMaxScaler()
x_train=scaler1.fit_transform(x_train)
x_test=scaler1.transform(x_test)
#划分633和96
x_train_633=x_train[:,0:628]
x_test_633=x_test[:,0:628]
x_train_96=x_train[:,628:724]
x_test_96=x_test[:,628:724]

In [ ]:
#读取模型数据
traindata=pd.read_csv(local_path+'\\blokagedata\\4191_sample_0123456789.csv',parse_dates=[0])
trainlabel=pd.read_csv(local_path+'\\blokagedata\\y_0123456789.csv',parse_dates=[0])
#改个名字
x=traindata#.drop(['32','205','375','408','414'],axis=1)
y=np.array(trainlabel).astype(int)
#划分训练集和测试集
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1,stratify=y)
#归一化
scaler1=MinMaxScaler()
x_train=scaler1.fit_transform(x_train)
x_test=scaler1.transform(x_test)

In [ ]:
#PCA
pca = PCA(n_components='mle')
x_train = pca.fit_transform(x_train)
x_test=pca.transform(x_test)
print(pca.explained_variance_ratio_)
# #SVD
# svd=TruncatedSVD(n_components=50)
# xx = svd.fit_transform(x_one)
# #NMF
# nmf=NMF(n_components=200)
# x_train = nmf.fit_transform(x_train)
# x_test=nmf.transform(x_test)
#归一化
scaler2=MinMaxScaler()
x_train=scaler2.fit_transform(x_train)
x_test=scaler2.transform(x_test)
print(x_train.shape)

# 回归拟合

## Regression

In [ ]:
reg = LinearRegression()
reg.fit(x_train_96,x_train_633)
score=reg.score(x_test_96,x_test_633)
print(score)
pre_train_633=reg.predict(x_train_96)
pre_test_633=reg.predict(x_test_96)
x1=np.concatenate((pre_train_633,x_train_96),axis=1)
x2=np.concatenate((pre_test_633,x_test_96),axis=1)

In [ ]:
reg = KNeighborsRegressor(n_neighbors=2)
reg.fit(x_train_96,x_train_633)
score=reg.score(x_test_96,x_test_633)
print(score)
pre_train_633=reg.predict(x_train_96)
pre_test_633=reg.predict(x_test_96)
x1=np.concatenate((pre_train_633,x_train_96),axis=1)
x2=np.concatenate((pre_test_633,x_test_96),axis=1)

In [ ]:
mlp = MLPRegressor(hidden_layer_sizes=(1000), activation='relu', tol=0.000001,
                   solver='adam', alpha=0.01, batch_size='auto', 
                   learning_rate='constant', learning_rate_init=0.001, 
                   power_t=0.5, max_iter=50, shuffle=True,verbose=True)
mlp.fit(x_train_96,x_train_633)
score=mlp.score(x_test_96,x_test_633)
print(score)

In [ ]:
x_train=x1
x_test=x2
print(x_train.shape)

## PCA

In [ ]:
#PCA
pca = PCA(n_components=200)
x_train = pca.fit_transform(x_train)
x_test=pca.transform(x_test)
# #SVD
# svd=TruncatedSVD(n_components=50)
# xx = svd.fit_transform(x_one)
# #NMF
# nmf=NMF(n_components=200)
# x_train = nmf.fit_transform(x_train)
# x_test=nmf.transform(x_test)
#归一化
scaler2=MinMaxScaler()
x_train=scaler2.fit_transform(x_train)
x_test=scaler2.transform(x_test)
print(x_train.shape)

# 分类

## SVM

In [ ]:
SVMclf=SVC(decision_function_shape='ovr',C=1000.0,gamma='auto',kernel='rbf')
SVMclf.fit(x_train,y_train.ravel())
score=SVMclf.score(x_test,y_test.ravel())
print(score)

## LogisticRegression

In [ ]:
LRclf=LogisticRegression(C=1.0,penalty='l1',solver='saga',multi_class='multinomial',max_iter=10000,n_jobs=-1)
LRclf.fit(x_train,y_train.ravel())
score=LRclf.score(x_test,y_test.ravel())
print(score)

In [ ]:
LRclf=LogisticRegression(C=1.0,penalty='l2',solver='saga',multi_class='multinomial',max_iter=1000,n_jobs=-1)
LRclf.fit(x_train,y_train.ravel())
score=LRclf.score(x_test,y_test.ravel())
print(score)

In [ ]:
LRclf=LogisticRegression(C=1.0,penalty='l2',solver='saga',multi_class='ovr',max_iter=1000,n_jobs=-1)
LRclf.fit(x_train,y_train.ravel())
score=LRclf.score(x_test,y_test.ravel())
print(score)

## SGD

In [ ]:
sgd=SGDClassifier(n_jobs=-1)
sgd.fit(x_train,y_train.ravel())
score=sgd.score(x_test,y_test.ravel())
print(score)

## GaussianProcess

In [ ]:
# GPC=GaussianProcessClassifier(n_jobs=-1)
# GPC.fit(x_train,y_train.ravel())
# score=GPC.score(x_test,y_test.ravel())
# print(score)

## KNN

In [ ]:
knn=KNeighborsClassifier(n_neighbors=1)
knn.fit(x_train,y_train.ravel())
score=knn.score(x_test,y_test.ravel())
print(score)

## Tree

In [ ]:
tr = tree.DecisionTreeClassifier()
tr.fit(x_train,y_train.ravel())
score=tr.score(x_test,y_test.ravel())
print(score)

## RandomForest

In [ ]:
rf = RandomForestClassifier(n_estimators=30,n_jobs=-1)
rf.fit(x_train,y_train.ravel())
score=rf.score(x_test,y_test.ravel())
print(score)

## LGBM

In [ ]:
# score=[]
# for n in range(10,100,5):
#     for i in range(10,100,5):
# #归一化
# scaler=MinMaxScaler()
# x_one=scaler.fit_transform(x)
# #PCA
# pca = PCA(n_components=n)
# xx = pca.fit_transform(x_one)
# #SVD
# svd=TruncatedSVD(n_components=50)
# xx = svd.fit_transform(x_one)
# #归一化
# scaler2=MinMaxScaler()
# x_one=scaler2.fit_transform(xx)
# #划分训练集和测试集
# x_train,x_test,y_train,y_test=train_test_split(x_one,y,test_size=0.2,random_state=1,stratify=y)

gbm = lgb.LGBMRegressor(objective='multiclass', num_leaves=100,max_depth=-1,num_class=10,boosting='gbdt',learning_rate=0.05, n_estimators=1000,nthread=-1)
gbm.fit(x_train, y_train.ravel(), eval_set=[(x_test, y_test.ravel())], early_stopping_rounds=5)
# 测试机预测
y_pred = gbm.predict(x_test,num_iteration=gbm.best_iteration_)
#     # 模型评估
print('The accuracy of prediction is:',accuracy_score(y_test,np.argmax(y_pred,axis=1)))
# # feature importances
# print('Feature importances:', list(gbm.feature_importances_))
# score.append([n,i,accuracy_score(y_test,np.argmax(y_pred,axis=1))])

## CatBoost

In [ ]:
cat = CatBoostClassifier(iterations=300, depth=10,learning_rate=0.05, loss_function='MultiClass',logging_level='Verbose')
cat.fit(x_train,y_train,eval_set=(x_test, y_test),plot=True)
score=cat.score(x_test,y_test.ravel())
print('score:',score)